In [6]:
import pandas as pd

file_path='/Users/kkkg0829/Desktop/project/data set/B0005_with_lowess_features.csv'
df=pd.read_csv(file_path)

# 기본 피처 (lowess 파생피처 사용 X)

In [5]:
import numpy as np
import plotly.express as px
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler



# 1. 사용할 feature 선택
features1 = [
    'Voltage_measured', 'Current_measured', 'Temperature_measured',
    'Current_load', 'Voltage_load'
]
X = df[features1]

# 2. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lof = LocalOutlierFactor(
        n_neighbors=30,
        contamination=0.005   # 예상 이상치 비율
    )

y_pred = lof.fit_predict(X_scaled)
    
lof_scores = -lof.negative_outlier_factor_

    # 결과를 데이터 프레임에 추가

df['lof_pred'] = y_pred
df['lof_scores'] = lof_scores
df['lof_anomaly'] = np.where(y_pred == -1, 1, 0)  # -1을 이상치(Anomaly)로 변환

print("LOF 이상치:", df['lof_anomaly'].sum())



LOF 이상치: 252


In [11]:
contamination=0.005 # 정한 이상치 비율
threshold = np.quantile(lof_scores, 1 - contamination)
print("LOF 임계값(threshold):", threshold)

LOF 임계값(threshold): 1.676333654209548


In [8]:
import plotly.express as px

cycle_df = df.groupby('cycle_idx')['lof_scores'].mean().reset_index()

fig = px.line(
    cycle_df,
    x='cycle_idx',
    y='lof_scores',      
    title='Average LOF Score per Cycle',
    markers=True
)
fig.show()

In [12]:
cycle_anom = df.groupby('cycle_idx')['lof_anomaly'].sum().reset_index()

fig = px.bar(
    cycle_anom,
    x='cycle_idx',
    y='lof_anomaly',
    title='Number of Anomalies per Cycle'
)
fig.show()


In [13]:
cycle_peak = df.groupby('cycle_idx')['lof_scores'].max().reset_index()

fig = px.line(
    cycle_peak,
    x='cycle_idx',
    y='lof_scores',
    title='Peak LOF Score per Cycle',
    markers=True
)
fig.show()